In [ ]:
import torch
%load_ext autoreload
%autoreload 2
print(torch.cuda.is_available(),torch.cuda.current_device())
# torch.cuda.device(1)
# torch.cuda.get_device_name(0)
data_path = "/data/desponds/data/Paraphrase"

In [ ]:
from preprocessing import preprocessing_paraphrasing
datasets, tokenized = preprocessing_paraphrasing()

In [ ]:
datasets['en']['train'][0], datasets['fr']['train'][9873]

In [ ]:
from training import get_trainers_paraphrasing
trainers = get_trainers_paraphrasing(tokenized)

In [ ]:
from training import get_models_paraphrasing
models = get_models_paraphrasing(trainers)

## Translation

In [ ]:
from translation import translate_fr_en_paraphrase

# Translate the test split of the french dataset
translated_fr_en = datasets['fr']['test'].map(translate_fr_en_paraphrase, batched=True, batch_size = 32)

# import pickle
# with open(f"{data_path}/translated_dataset.pickle", 'wb') as handle:
#     pickle.dump(translated_fr_en, handle)

In [ ]:
import pickle
with open(f"{data_path}/translated_dataset.pickle", 'rb') as handle2:
    translated_fr_en = pickle.load(handle2)

In [ ]:
tokenized_fr_en = translated_fr_en.map(lambda ex : preprocess(ex,'en'))

## Evaluation

### Dataset FR with CamemBERT

In [ ]:
import numpy as np
logits,labels, test = trainers['fr'].predict(tokenized['fr']['test'])
print(test) 

### Dataset EN with RoBERTa

In [ ]:
logits,labels, test = trainers['en'].predict(tokenized['en']['test'])
print(test) 

### Dataset FR with RoBERTa

In [ ]:
import numpy as np
logits,labels, test = trainers['en'].predict(tokenized_fr_en)
print(test)

In [ ]:
logits,labels, test = trainers['en'].predict(tokenized['fr']['test'])
print(test) #{'test_loss': 1.7237563133239746, 'test_accuracy': 0.5955, 'test_runtime': 10.9047, 'test_samples_per_second': 183.408, 'test_steps_per_second': 5.777}

## Results

In [ ]:
import pandas as pd
data = {
    'task' : ['Paraphrase', 'Paraphrase', 'Paraphrase', 'Paraphrase'],
    'model'   : ['RoBERTa', 'RoBERTa', 'RoBERTa', 'CamemBERT'],
    'train_dataset' : [ 'paws-x_en', 'paws-x_en', 'paws-x_en', 'paws-x_fr'],
    'nb_sample_train' : [49401,49401,49401, 49101],
    'test_dataset' : ['paws-x_en', 'paws-x_fr', 'paws-x_fr_translated', 'paws-x_fr'],
    'translated' : ['no', 'no', 'yes', 'no'],
    'accuracy'    : [ 0.9465, 0.5955,  0.9110, 0.8990 ]
}
results = pd.DataFrame(data)
results